### Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from masserstein import NMRSpectrum, estimate_proportions
import matplotlib.pyplot as plt
import pulp
import pickle

### Data

In [ ]:
experiment_name = 'PMG 287 monitoring with full mixture spectrum'

In [ ]:
mixture_separator = '\t'

if experiment_name == 'PMG 287 monitoring':
    whole_mixture_or_regions = 'regions'
elif experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    whole_mixture_or_regions = 'whole'

if whole_mixture_or_regions == 'regions':
    data_path = '../../data/PMG_287/'
elif whole_mixture_or_regions == 'whole':
    data_path = '../../data/PMG_287_full_mixture_spectrum/'
    
mixture_path = data_path + 'PMG_287_seria1.csv'

#### Mixture in time

In [ ]:
mixture_time_data = pd.read_csv(mixture_path, sep = mixture_separator).iloc[:,:-1]

In [ ]:
names = ['ppm'] + ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]

In [ ]:
mixture_time_data.columns = names

In [ ]:
def load_spectrum(mixture_time_data, moment_of_time):
    ppm = mixture_time_data['ppm']
    intensity = mixture_time_data['t'+str(moment_of_time)]
    sp = NMRSpectrum(confs = list(zip(ppm, intensity)))
    return sp

In [ ]:
%matplotlib notebook

In [ ]:
plt.gca().invert_xaxis()
NMRSpectrum.plot_all([load_spectrum(mixture_time_data, 1),
                     load_spectrum(mixture_time_data, 1000)], profile=True)

#### Reagents

In [ ]:
# Integral(3.343730, 3.256018) product
# Integral(3.722376, 3.681636) product
# Integral(3.756886, 3.722376) product
# Integral(4.078496, 3.994139) product

# Integral(3.406997, 3.343730) silane
# Integral(3.832615, 3.756886) silane
# Integral(6.564104, 6.422073) silane

# Integral(3.679897, 3.599926) hexene
# Integral(4.129027, 4.079348) hexene
# Integral(4.857709, 4.668059) hexene
# Integral(7.838666, 7.616258) hexene
# Integral(8.647265, 8.440374) hexene

# Integral(10.387961, 2.952301) all

In [ ]:
reagents_sp = []

mix_to_cut = pd.DataFrame(load_spectrum(mixture_time_data, 250).confs)

#baseline correction
#mix_to_cut.iloc[:,1] = mix_to_cut.iloc[:, 1] + 0.02

#hexene
reagent0 = mix_to_cut[mix_to_cut[0].apply(lambda x: (x>3.599926 and x<3.679897) or
                                                      (x>4.079348 and x<4.129027) or 
                                                      (x>4.668059 and x<4.857709) or 
                                                      (x>7.616258 and x<7.838666) or 
                                                      (x>8.440374 and x<8.647265)
                                         )
                     ]

ppm = reagent0.iloc[:,0]
ints = reagent0.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))



#triethylsilane                           
reagent1 = mix_to_cut[mix_to_cut[0].apply(lambda x: (x>3.343730 and x<3.406997) or
                                                      (x>3.756886 and x<3.832615) or 
                                                      (x>6.422073 and x<6.564104)
                                         )
                     ]

ppm = reagent1.iloc[:,0]
ints = reagent1.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))

#product                              
reagent2 = mix_to_cut[mix_to_cut[0].apply(lambda x: (x>3.256018 and x<3.343730) or
                                                      (x>3.681636 and x<3.722376) or 
                                                      (x>3.722376 and x<3.756886) or
                                                      (x>3.994139 and x<4.078496)
                                         )
                     ]

ppm = reagent2.iloc[:,0]
ints = reagent2.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))

In [ ]:
plt.gca().invert_xaxis()

load_spectrum(mixture_time_data, 250).plot(profile=True)
NMRSpectrum.plot_all(reagents_sp, profile=True)

#### Mixture

In [ ]:
if whole_mixture_or_regions == 'whole':
    
    mixture_time_data = mixture_time_data[mixture_time_data.iloc[:,0].apply(lambda x: x>2.952301 and x<10.387961)]
    
elif whole_mixture_or_regions == 'regions':
    
    mixture_time_data = mixture_time_data[mixture_time_data.iloc[:,0].apply(lambda x:

                                                                (x>3.599926 and x<3.679897) or
                                                                (x>4.079348 and x<4.129027) or 
                                                                (x>4.668059 and x<4.857709) or 
                                                                (x>7.616258 and x<7.838666) or 
                                                                (x>8.440374 and x<8.647265) or

                                                                (x>3.343730 and x<3.406997) or
                                                                (x>3.756886 and x<3.832615) or 
                                                                (x>6.422073 and x<6.564104) or

                                                                (x>3.256018 and x<3.343730) or
                                                                (x>3.681636 and x<3.722376) or 
                                                                (x>3.722376 and x<3.756886) or
                                                                (x>3.994139 and x<4.078496)
                                                                           )
                                         ]

In [ ]:
plt.gca().invert_xaxis()

load_spectrum(mixture_time_data, 250).plot(profile=True)
NMRSpectrum.plot_all(reagents_sp, profile=True)

### Saving preprocessed spectra

In [ ]:
# np.savetxt(data_path + 'preprocessed_hexene.csv', reagent0, delimiter = '\t')
# np.savetxt(data_path + 'preprocessed_triethylsilane.csv', reagent1, delimiter = '\t')
# np.savetxt(data_path + 'preprocessed_product.csv', reagent2, delimiter = '\t')

# if whole_mixture_or_regions == 'whole':
#     np.savetxt(data_path + 'preprocessed_mixture_whole.csv', mixture_time_data, delimiter = '\t')
    
# elif whole_mixture_or_regions == 'regions':
#     np.savetxt(data_path + 'preprocessed_mixture_cut_to_regions.csv', mixture_time_data, delimiter = '\t')